# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps
import json
from pprint import pprint

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_csv = "../output_data/city_weather.csv"
city_weather_df = pd.read_csv(city_weather_csv)
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Edmundston,47.3737,-68.3251,51.13,83,56,8.66,CA,1631358598
1,Ramon Magsaysay,8.0019,123.5094,75.15,92,92,3.13,PH,1631358618
2,Az Zabadānī,33.7249,36.1005,81.21,32,0,10.36,SY,1631358619
3,Fortuna,40.5982,-124.1573,52.72,75,75,0.00,US,1631358620
4,Saint-Alban,43.7000,1.4167,77.09,64,20,3.44,FR,1631358620
...,...,...,...,...,...,...,...,...,...
592,Wildeshausen,52.9000,8.4333,68.11,88,56,13.38,DE,1631359066
593,Estaires,50.6457,2.7278,69.89,78,100,13.42,FR,1631359067
594,Saguday,16.5428,121.5638,75.18,96,100,2.84,PH,1631359067
595,Gulu,2.7746,32.2990,79.50,60,38,2.19,UG,1631359068


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations, store humidity data in humidity
locations = city_weather_df[["Lat", "Lng"]]
humidity = city_weather_df["Humidity"]

In [4]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
# The highest humidity for this dataframe is 100, set max_intensity to 100
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70
narrow_cities = city_weather_df.loc[city_weather_df["Max Temp"]>=70]
narrow_cities = narrow_cities.loc[narrow_cities["Max Temp"]<80]
# Wind speed less than 10 mph.
narrow_cities = narrow_cities.loc[narrow_cities["Wind Speed"]<=10]
# Zero cloudiness.
narrow_cities = narrow_cities.loc[narrow_cities["Cloudiness"]==0]
# Drop any rows that don't contain all three conditions
narrow_cities.dropna(inplace=False)
# Reset dataframe index
narrow_cities = narrow_cities.reset_index(drop=True)
# view cities info for indeal weather condition
narrow_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ayvacık,39.6011,26.4047,77.52,30,0,6.73,TR,1631358623
1,Piastów,52.1843,20.8395,77.05,47,0,9.22,PL,1631358627
2,Topola,44.2542,20.6825,79.88,17,0,6.98,RS,1631358649
3,Lagodekhi,41.8268,46.2767,79.75,46,0,5.30,GE,1631358708
4,Leavenworth,39.2333,-95.0336,74.25,80,0,1.99,US,1631358712
5,Escoublac,47.2947,-2.3596,70.97,61,0,3.44,FR,1631358721
6,Uiwang,37.3449,126.9690,76.32,75,0,3.27,KR,1631358777
7,Buchrain,47.0860,8.3462,77.00,65,0,0.00,CH,1631358805
8,Herbignac,47.4482,-2.3181,71.19,60,0,3.44,FR,1631358807
9,Pornichet,47.2668,-2.3379,71.29,61,0,3.44,FR,1631358809


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Google api url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params dictionary to update each iteration
params = {
    "radius": 5000 ,
    "keyword": "hotel",
    "key": g_key
}

for index,row in narrow_cities.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # get hotel response and convert to json
    hotel_response = requests.get(base_url, params=params).json()
    # pprint(hotel_response)
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        # return the hotel name back to dataframe
        narrow_cities.loc[index, "Hotel Name"] = hotel_response["results"][0]["name"]
        # return hotel's lat and lng 
        narrow_cities.loc[index, "Hotel Lat"] = hotel_response["results"][0]["geometry"]["location"]["lat"]
        narrow_cities.loc[index, "Hotel Lng"] = hotel_response["results"][0]["geometry"]["location"]["lng"]       
    except (KeyError, IndexError):
        # if no hotel found within 5000 meters, return a sentence to tell
        narrow_cities.loc[index, "Hotel Name"] = "*Sorry, no hotel found within 5000 meters.*"
        print("Missing field/result... skipping.")

# Visualize to confirm lat lng appear
# visual_narrow_cities = narrow_cities.iloc[:,0:10]
# visual_narrow_cities
narrow_cities

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
0,Ayvacık,39.6011,26.4047,77.52,30,0,6.73,TR,1631358623,"*Sorry, no hotel found within 5000 meters.*",NaN,NaN
1,Piastów,52.1843,20.8395,77.05,47,0,9.22,PL,1631358627,HOTEL Venecia Palace,52.152878,20.888722
2,Topola,44.2542,20.6825,79.88,17,0,6.98,RS,1631358649,Хотел „Опленац“ ДОО,44.247496,20.680574
3,Lagodekhi,41.8268,46.2767,79.75,46,0,5.30,GE,1631358708,Royal Palace,41.841913,46.283009
4,Leavenworth,39.2333,-95.0336,74.25,80,0,1.99,US,1631358712,"*Sorry, no hotel found within 5000 meters.*",NaN,NaN
5,Escoublac,47.2947,-2.3596,70.97,61,0,3.44,FR,1631358721,Hôtel Eco Nuit Guérande,47.337803,-2.422264
6,Uiwang,37.3449,126.9690,76.32,75,0,3.27,KR,1631358777,Hotel-On,37.380262,126.974552
7,Buchrain,47.0860,8.3462,77.00,65,0,0.00,CH,1631358805,Grand Hotel National Luzern,47.054896,8.315736
8,Herbignac,47.4482,-2.3181,71.19,60,0,3.44,FR,1631358807,Hôtel Les Brières,47.449868,-2.317252
9,Pornichet,47.2668,-2.3379,71.29,61,0,3.44,FR,1631358809,Hôtel Escale Oceania Pornichet-La Baule,47.260896,-2.340285


In [7]:
# NOTE: Do not change any of the code in this cell
# to select all countries with hotels found within 5000 meters, save in hotel_df
hotel_df = narrow_cities.dropna(inplace=False)
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]


In [8]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

# Create heat layer
# The highest humidity for this dataframe is 100, set max_intensity to 100
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)
# Add marker
markers = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))